In [1]:
import tensorflow as tf 
import numpy as np
import pandas as pd
from glob import glob
import os
import mne 
import matplotlib.pyplot as plt

In [2]:
all_files_path=glob('epilepsy_data/*.edf')
print(len(all_files_path))

28


In [3]:
all_files_path[0]

'epilepsy_data\\h01.edf'

In [4]:
healthy_file_path=[i for i in all_files_path if  'h' in i.split('\\')[1]]
patient_file_path=[i for i in all_files_path if  's' in i.split('\\')[1]]

In [5]:
def read_data(file_path):
    datax=mne.io.read_raw_edf(file_path,preload=True)
    datax.set_eeg_reference()
    datax.filter(l_freq=1,h_freq=45)
    epochs=mne.make_fixed_length_epochs(datax,duration=25,overlap=0)
    epochs=epochs.get_data()
    return epochs #trials,channel,length

In [6]:
data=read_data(healthy_file_path[0])

Extracting EDF parameters from c:\Users\danus\Downloads\epilepsy_dl\epilepsy_data\h01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 825 samples (3.300 s)

Not setting metadata
37 matching events found
No baseline correction applied
0 projection items

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


In [7]:
data.shape

(37, 19, 6250)

In [8]:
%%capture
control_epochs_array=[read_data(subject) for subject in healthy_file_path]
patients_epochs_array=[read_data(subject) for subject in patient_file_path]

In [9]:
control_epochs_labels=[len(i)*[0] for i in control_epochs_array]
patients_epochs_labels=[len(i)*[1] for i in patients_epochs_array]
print(len(control_epochs_labels),len(patients_epochs_labels))

14 14


In [10]:
data_list=control_epochs_array+patients_epochs_array
label_list=control_epochs_labels+patients_epochs_labels
print(len(data_list),len(label_list))

28 28


In [11]:
groups_list=[[i]*len(j) for i, j in enumerate(data_list)]

In [12]:
#groups_list

In [13]:
data_array=np.vstack(data_list)
label_array=np.hstack(label_list)
group_array=np.hstack(groups_list)
data_array=np.moveaxis(data_array,1,2)

print(data_array.shape,label_array.shape,group_array.shape)

(1142, 6250, 19) (1142,) (1142,)


In [14]:
from tensorflow.keras.layers import Conv1D,BatchNormalization,BatchNormalization,LeakyReLU,Dropout,MaxPool1D,Flatten,Dense,GlobalAveragePooling1D,AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [15]:
def cnn():
    model = Sequential()
    model.add(Conv1D(filters = 5,kernel_size =3,strides =1,input_shape=(6250,19)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))
    model.add(Conv1D(filters = 5,kernel_size =3,strides =1))
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters = 5,kernel_size =3,strides =1))
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters = 5,kernel_size =3,strides =1))
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))
    model.add(Conv1D(filters = 5,kernel_size =3,strides =1))
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling1D())
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile('adam',loss= 'binary_crossentropy',metrics=['accuracy'])
    return model
    


In [16]:
model = cnn()

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 6248, 5)           290       
                                                                 
 batch_normalization (Batch  (None, 6248, 5)           20        
 Normalization)                                                  
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 6248, 5)           0         
                                                                 
 max_pooling1d (MaxPooling1  (None, 3124, 5)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 3122, 5)           80        
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 3122, 5)           0

In [18]:
from sklearn.model_selection import GroupKFold,LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler

In [19]:
gkf=GroupKFold()

In [20]:
accuracy = []

In [21]:
for train_index,val_index in gkf.split(data_array,label_array,groups=group_array):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler()
    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    model=cnn()
    model.fit(train_features,train_labels,epochs=50,batch_size=128,validation_data=(val_features,val_labels))
    accuracy.append(model.evaluate(val_features,val_labels)[1])


Epoch 1/50
8/8 [==============================] - 4s 252ms/step - loss: 0.6992 - accuracy: 0.4863 - val_loss: 0.6936 - val_accuracy: 0.5498
Epoch 2/50
8/8 [==============================] - 2s 198ms/step - loss: 0.6947 - accuracy: 0.4566 - val_loss: 0.6870 - val_accuracy: 0.6883
Epoch 3/50
8/8 [==============================] - 2s 227ms/step - loss: 0.6926 - accuracy: 0.5093 - val_loss: 0.6834 - val_accuracy: 0.6840
Epoch 4/50
8/8 [==============================] - 2s 221ms/step - loss: 0.6916 - accuracy: 0.5291 - val_loss: 0.6784 - val_accuracy: 0.6840
Epoch 5/50
8/8 [==============================] - 2s 235ms/step - loss: 0.6909 - accuracy: 0.5192 - val_loss: 0.6746 - val_accuracy: 0.6840
Epoch 6/50
8/8 [==============================] - 2s 245ms/step - loss: 0.6898 - accuracy: 0.5170 - val_loss: 0.6711 - val_accuracy: 0.6840
Epoch 7/50
8/8 [==============================] - 2s 238ms/step - loss: 0.6884 - accuracy: 0.5170 - val_loss: 0.6665 - val_accuracy: 0.6840
Epoch 8/50
8/8 [====

In [22]:
accuracy

[0.7532467246055603,
 0.4757281541824341,
 0.5170940160751343,
 0.5466101765632629,
 0.5957446694374084]

In [23]:
from joblib import dump,load
from tensorflow.keras.models import save_model

In [24]:
!pip install h5py


In [ ]:
model.save_model('eeg_model.h5')